In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
from pathlib import Path
import sys

cwd = Path.cwd().resolve()

# Walk upwards until we find SpotifyAzureProject
for p in [cwd] + list(cwd.parents):
    if p.name == "Spotify-Data-Engineering-Project":
        project_root = p
        break
else:
    raise RuntimeError("SpotifyAzureProject not found in path tree")

print("Resolved project root:", project_root)

sys.path.insert(0, str(project_root))
print(sys.path[:3])


In [0]:
from scripts import transformations

### DimUser

### **AUTOLOADER**


In [0]:
df_user = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimUser/checkpoint")\
    .load("abfss://bronze@storageazurepr.dfs.core.windows.net/DimUser")

In [0]:
display(df_user)

In [0]:
# Transformation's

df = df_user.withColumn("user_name",upper(col("user_name")))
display(df)

In [0]:
df_user_obj = transformations.resuable()

df_user = df_user_obj.dropColumns(df_user,['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])
display(df_user)

In [0]:
display(df_user)

In [0]:
df_user.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimUser/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storageazurepr.dfs.core.windows.net/DimUser/data")\
    .toTable("spotify_catalog.silver.DimUser")

### Dim Artist

In [0]:
df_art = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimArtist/checkpoint")\
    .load("abfss://bronze@storageazurepr.dfs.core.windows.net/DimArtist")

In [0]:
display(df_art)

In [0]:
df_art_obj = transformations.resuable()

df_art = df_art_obj.dropColumns(df_art,['_rescued_data'])
df_art = df_art.dropDuplicates(['artist_id'])
display(df_art)

In [0]:
df_art.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimArtist/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storageazurepr.dfs.core.windows.net/DimArtist/data")\
    .toTable("spotify_catalog.silver.DimArtist")

### DimTrack

In [0]:
df_track = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimTrack/checkpoint")\
    .load("abfss://bronze@storageazurepr.dfs.core.windows.net/DimTrack")

In [0]:
display(df_track)

In [0]:
df_track = df_track.withColumn("durationFlag",when(df_track.duration_sec<150,"Low").when(df_track.duration_sec<300,"Medium").otherwise("High"))
df_track = df_track.withColumn("track_name",regexp_replace(col('track_name'),'-',' '))
display(df_track)

In [0]:
df_track = transformations.resuable().dropColumns(df_track,['_rescued_data'])

In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimTrack/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storageazurepr.dfs.core.windows.net/DimTrack/data")\
    .toTable("spotify_catalog.silver.DimTrack")

### DimDate

In [0]:
df_date = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimDate/checkpoint")\
    .load("abfss://bronze@storageazurepr.dfs.core.windows.net/DimDate")

In [0]:
display(df_date)
df_date = transformations.resuable().dropColumns(df_date,['_rescued_data'])

In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/DimDate/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storageazurepr.dfs.core.windows.net/DimDate/data")\
    .toTable("spotify_catalog.silver.DimDate")

### FactStream

In [0]:
df_fact = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/FactStream/checkpoint")\
    .load("abfss://bronze@storageazurepr.dfs.core.windows.net/FactStream")

In [0]:
display(df_fact)

In [0]:
df_fact = transformations.resuable().dropColumns(df_fact,['_rescued_data'])


In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storageazurepr.dfs.core.windows.net/FactStream/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storageazurepr.dfs.core.windows.net/FactStream/data")\
    .toTable("spotify_catalog.silver.FactStream")